In [4]:
# This runs in viola for web application
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType
import ipywidgets as widgets
from IPython.display import display

# Load data
data = pd.read_csv('cuisines.csv')
X = data.iloc[:, 2:]
y = data['cuisine']

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

# Optional: scale for better performance
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train SVC
model = SVC(kernel='linear', C=10, probability=True, random_state=0)
model.fit(X_train_scaled, y_train)

# Create ONNX model
initial_type = [('input', FloatTensorType([None, X.shape[1]]))]
onnx_model = convert_sklearn(model, initial_types=initial_type)
with open("svc_cuisine_model.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())


# Widgets for Interactive App
input_widgets = []
feature_names = X.columns

for feature in feature_names:
    input_widgets.append(widgets.FloatText(description=feature, value=0.0))

predict_button = widgets.Button(description="Predict Cuisine")
output = widgets.Output()

def on_predict_clicked(b):
    input_data = pd.DataFrame([[w.value for w in input_widgets]], columns=feature_names)
    input_scaled = scaler.transform(input_data)
    prediction = model.predict(input_scaled)[0]
    probas = model.predict_proba(input_scaled)

    with output:
        output.clear_output()
        print(f"✅ Predicted Cuisine: {prediction}")
        print("\n🔢 Probabilities:")
        for label, prob in zip(model.classes_, probas[0]):
            print(f"{label}: {prob:.4f}")

predict_button.on_click(on_predict_clicked)

# Display app
display(widgets.VBox(input_widgets + [predict_button, output]))